# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,May 23 2023 12:54PM,263497,19,MSP222060,"Methapharm, Inc.",Released
1,May 23 2023 12:31PM,263496,19,GP05242023,"Granules Pharmaceuticals, Inc.",Released
2,May 23 2023 12:25PM,263495,15,9509052,"Emersa Waterbox, LLC",Released
3,May 23 2023 12:20PM,263494,15,9509051,"Emersa Waterbox, LLC",Released
4,May 23 2023 12:16PM,263493,10,MSP222073,"Methapharm, Inc.",Released
5,May 23 2023 12:16PM,263493,10,MSP222051,"Methapharm, Inc.",Released
6,May 23 2023 12:16PM,263493,10,MSP222074,"Methapharm, Inc.",Released
7,May 23 2023 12:16PM,263493,10,MSP222075,"Methapharm, Inc.",Released
8,May 23 2023 12:16PM,263493,10,MSP222076,"Methapharm, Inc.",Released
9,May 23 2023 12:16PM,263493,10,MSP222077,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
33,263493,Released,7
34,263494,Released,1
35,263495,Released,1
36,263496,Released,1
37,263497,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
263493,NaN,NaN,7.0
263494,NaN,NaN,1.0
263495,NaN,NaN,1.0
263496,NaN,NaN,1.0
263497,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
263302,0.0,2.0,1.0
263409,0.0,8.0,18.0
263421,0.0,4.0,3.0
263439,12.0,8.0,3.0
263441,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
263302,0,2,1
263409,0,8,18
263421,0,4,3
263439,12,8,3
263441,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,263302,0,2,1
1,263409,0,8,18
2,263421,0,4,3
3,263439,12,8,3
4,263441,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,263302,,2,1
1,263409,,8,18
2,263421,,4,3
3,263439,12,8,3
4,263441,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,May 23 2023 12:54PM,263497,19,"Methapharm, Inc."
1,May 23 2023 12:31PM,263496,19,"Granules Pharmaceuticals, Inc."
2,May 23 2023 12:25PM,263495,15,"Emersa Waterbox, LLC"
3,May 23 2023 12:20PM,263494,15,"Emersa Waterbox, LLC"
4,May 23 2023 12:16PM,263493,10,"Methapharm, Inc."
11,May 23 2023 12:13PM,263492,10,"Methapharm, Inc."
12,May 23 2023 12:13PM,263491,10,"Methapharm, Inc."
13,May 23 2023 12:10PM,263489,10,Emerginnova
15,May 23 2023 12:01PM,263488,20,VITABIOTICS USA INC.
16,May 23 2023 11:38AM,263483,15,"Brookfield Pharmaceuticals, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,May 23 2023 12:54PM,263497,19,"Methapharm, Inc.",,,1
1,May 23 2023 12:31PM,263496,19,"Granules Pharmaceuticals, Inc.",,,1
2,May 23 2023 12:25PM,263495,15,"Emersa Waterbox, LLC",,,1
3,May 23 2023 12:20PM,263494,15,"Emersa Waterbox, LLC",,,1
4,May 23 2023 12:16PM,263493,10,"Methapharm, Inc.",,,7
5,May 23 2023 12:13PM,263492,10,"Methapharm, Inc.",,,1
6,May 23 2023 12:13PM,263491,10,"Methapharm, Inc.",,,1
7,May 23 2023 12:10PM,263489,10,Emerginnova,,,2
8,May 23 2023 12:01PM,263488,20,VITABIOTICS USA INC.,,,1
9,May 23 2023 11:38AM,263483,15,"Brookfield Pharmaceuticals, LLC",,3,4


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 23 2023 12:54PM,263497,19,"Methapharm, Inc.",1,,
1,May 23 2023 12:31PM,263496,19,"Granules Pharmaceuticals, Inc.",1,,
2,May 23 2023 12:25PM,263495,15,"Emersa Waterbox, LLC",1,,
3,May 23 2023 12:20PM,263494,15,"Emersa Waterbox, LLC",1,,
4,May 23 2023 12:16PM,263493,10,"Methapharm, Inc.",7,,
5,May 23 2023 12:13PM,263492,10,"Methapharm, Inc.",1,,
6,May 23 2023 12:13PM,263491,10,"Methapharm, Inc.",1,,
7,May 23 2023 12:10PM,263489,10,Emerginnova,2,,
8,May 23 2023 12:01PM,263488,20,VITABIOTICS USA INC.,1,,
9,May 23 2023 11:38AM,263483,15,"Brookfield Pharmaceuticals, LLC",4,3,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 23 2023 12:54PM,263497,19,"Methapharm, Inc.",1,,
1,May 23 2023 12:31PM,263496,19,"Granules Pharmaceuticals, Inc.",1,,
2,May 23 2023 12:25PM,263495,15,"Emersa Waterbox, LLC",1,,
3,May 23 2023 12:20PM,263494,15,"Emersa Waterbox, LLC",1,,
4,May 23 2023 12:16PM,263493,10,"Methapharm, Inc.",7,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 23 2023 12:54PM,263497,19,"Methapharm, Inc.",1.0,NaN,NaN
1,May 23 2023 12:31PM,263496,19,"Granules Pharmaceuticals, Inc.",1.0,NaN,NaN
2,May 23 2023 12:25PM,263495,15,"Emersa Waterbox, LLC",1.0,NaN,NaN
3,May 23 2023 12:20PM,263494,15,"Emersa Waterbox, LLC",1.0,NaN,NaN
4,May 23 2023 12:16PM,263493,10,"Methapharm, Inc.",7.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 23 2023 12:54PM,263497,19,"Methapharm, Inc.",1.0,0.0,0.0
1,May 23 2023 12:31PM,263496,19,"Granules Pharmaceuticals, Inc.",1.0,0.0,0.0
2,May 23 2023 12:25PM,263495,15,"Emersa Waterbox, LLC",1.0,0.0,0.0
3,May 23 2023 12:20PM,263494,15,"Emersa Waterbox, LLC",1.0,0.0,0.0
4,May 23 2023 12:16PM,263493,10,"Methapharm, Inc.",7.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2634761,25.0,1.0,0.0
15,2107562,59.0,59.0,0.0
19,1580771,27.0,36.0,38.0
20,263488,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2634761,25.0,1.0,0.0
1,15,2107562,59.0,59.0,0.0
2,19,1580771,27.0,36.0,38.0
3,20,263488,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,25.0,1.0,0.0
1,15,59.0,59.0,0.0
2,19,27.0,36.0,38.0
3,20,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,25.0
1,15,Released,59.0
2,19,Released,27.0
3,20,Released,1.0
4,10,Executing,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,19,20
Status,,,,
Completed,0.0,0.0,38.0,0.0
Executing,1.0,59.0,36.0,0.0
Released,25.0,59.0,27.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,19,20
0,Completed,0.0,0.0,38.0,0.0
1,Executing,1.0,59.0,36.0,0.0
2,Released,25.0,59.0,27.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,19,20
0,Completed,0.0,0.0,38.0,0.0
1,Executing,1.0,59.0,36.0,0.0
2,Released,25.0,59.0,27.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()